# Sentiment Analysis in Nepali Language

This google colab is about sentiment analysis in Nepali language by fine-tuning BERT-derivative model. The dataset used in this notebook is mainly from [here](https://github.com/oya163/nepali-sentiment-analysis/blob/master/data/nepcls/csv/ss_ac_at_txt_unbal.csv)

## Installation

In [ ]:
!python3 -m pip install -U huggingface_hub
!python3 -m pip install -U transformers
!python3 -m pip install -U datasets evaluate
!python3 -m pip install -U accelerate
!python3 -m pip install -U seqeval

In [ ]:
!python3 -m pip install -U wandb

In [ ]:
# Wrap the text in ipython notebook
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

import warnings
warnings.filterwarnings("ignore")

# Data Preprocessing

## Prepare NepSA dataset

This is related to creating a dataset based off of the raw dataset from [Nepali Sentiment Analysis](https://raw.githubusercontent.com/oya163/nepali-sentiment-analysis/master/data/nepcls/csv/ss_ac_at_txt_unbal.csv) project

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# filepath = '/content/drive/MyDrive/nepsa_data/nepsa'

In [ ]:
# !wget https://raw.githubusercontent.com/oya163/nepali-sentiment-analysis/master/data/nepcls/csv/ss_ac_at_txt_bal.csv

In [ ]:
# import matplotlib.pyplot as plt
# import os
# import pandas as pd
# import torch
# import numpy as np
# pd.set_option('display.max_colwidth', None)

In [ ]:
# filepath = "/kaggle/working/ss_ac_at_txt_bal.csv"
# df = pd.read_csv(filepath,
#                    names=["Severity", "Category", "Aspect Word", "text"])

In [ ]:
# df.head()

In [ ]:
# df[df['Category']=='PROFANITY']

In [ ]:
# df['Category'].unique()

In [ ]:
# df = df[~df['Category'].isin(['FEEDBACK'])]
# df['Category'].unique()

In [ ]:
# def create_label(row):
#     if row['Category'] == "GENERAL" and row['Severity'] == 0:
#         return 0
#     elif row['Category'] == "GENERAL" and row['Severity'] == 1:
#         return 1
#     elif row['Category'] == "PROFANITY":
#         return 2
#     elif row['Category'] == "VIOLENCE":
#         return 3

# df['label'] = df.apply(create_label, axis=1).astype(int)

# df = df.drop(['Severity', 'Category', 'Aspect Word'], axis=1)
# df.head()


In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=163)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=163)


In [ ]:
# def create_csv(X, y, filename):
#     df = pd.DataFrame()
#     df['text'] = X
#     df['label'] = y
#     df.to_csv(f"{filename}.txt", sep='\t', header=False, index=False)

# create_csv(X_train, y_train, 'train')
# create_csv(X_val, y_val, 'valid')
# create_csv(X_test, y_test, 'test')


## Load NepSA dataset

In [ ]:
import os
from datasets import load_dataset

filepath="/kaggle/input/nepsa-data"
data_files = {
    "train": os.path.join(filepath, "train.txt"),
    "validation": os.path.join(filepath, "valid.txt"),
    "test": os.path.join(filepath, "test.txt"),
}

raw_datasets = load_dataset(os.path.join(filepath, "load_sa.py"), data_files=data_files)

Check the basic information on the loaded dataset

In [ ]:
raw_datasets

Check the data statistics

In [ ]:
from collections import Counter

for k, v in raw_datasets.items():
    print(k, Counter(raw_datasets[k]['label']))

In [ ]:
train_data = raw_datasets['train']
test_data = raw_datasets['test']
valid_data = raw_datasets['validation']

Check sample of tokens from train dataset

In [ ]:
print(train_data[10]["text"])

Check the NER tags (its IDS) of the corresponding sample

In [ ]:
print(raw_datasets["train"][10]["label"])

In [ ]:
ner_feature = raw_datasets["train"].features["label"]
ner_feature.num_classes

## Tokenization

In [ ]:
from transformers import AutoTokenizer

# model_checkpoint = "NepBERTa/NepBERTa"
model_checkpoint = "Rajan/NepaliBERT"
# model_checkpoint = "Rajan/nepbertaTorch"
# model_checkpoint = "Sakonii/distilbert-base-nepali"
# model_checkpoint = "xlm-roberta-large"

# model_checkpoint = "/kaggle/input/nepsa-model/model"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Data Preprocessing

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=False)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)
tokenized_val = valid_data.map(preprocess_function, batched=True)

# Fine Tuning

## Data Collation

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Load pre-trained model

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=4,
    from_tf=True if model_checkpoint=="NepBERTa/NepBERTa" else False
)

## Setup Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from datasets import load_metric

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    report = classification_report(y_true=labels, y_pred=predictions, output_dict=True)

    accuracy = report['accuracy']
    recall = report['weighted avg']['recall']
    precision = report['weighted avg']['precision']
    f1 = report['weighted avg']['f1-score']
    return {
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1": f1
    }



In [ ]:
model.config.num_labels

## Training

In [ ]:
# from google.colab import userdata
# from huggingface_hub import login, notebook_login

# login(token=userdata.get('hugging_face'))

In [ ]:
from transformers import TrainingArguments, Trainer

model_name = "nepsa"

args = TrainingArguments(
    model_name,
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=4,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
from kaggle_secrets import UserSecretsClient
import wandb

user_secrets = UserSecretsClient()
wandb_secret = user_secrets.get_secret("wandb")
wandb.login(key=wandb_secret)

trainer.train()

In [ ]:
trainer.evaluate()

## Save the model

In [ ]:
saved_model_path='nepsa'
trainer.save_model(saved_model_path)

## Evaluation

In [ ]:
predictions = trainer.predict(tokenized_test)

In [ ]:
import pandas as pd
final_predictions = np.argmax(predictions.predictions, axis=1)

label_map = {
    0: 'GENERAL POSITIVE',
    1: 'GENERAL NEGATIVE',
    2: 'PROFANITY',
    3: 'VIOLENCE'
}

prediction_data = []
for text, gt, pt in zip(tokenized_test['text'], tokenized_test['label'], final_predictions):
    prediction_data.append([text, gt, pt])
    
prediction_df = pd.DataFrame(prediction_data, columns=['text', 'ground_truth', 'predictions'])

In [ ]:
# prediction_df[prediction_df['predictions']==0]

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_true=prediction_df['ground_truth'], y_pred=prediction_df['predictions'])
print(report)

In [ ]:
from tabulate import tabulate

metrics = ['precision', 'recall', 'f1', 'accuracy']
prediction_results = []

for key, val in predictions.metrics.items():
    if any(item in key for item in metrics):
        prediction_results.append([key, str(round(val,4)*100)+'%'])

print(tabulate(prediction_results, headers=['Metric', 'Score']))

## Inference

In [ ]:
from transformers import pipeline

text_classifier = pipeline("text-classification", model=saved_model_path)

In [ ]:
inference_sentences = ["वा जैनु जि तपाइलाइ धन्यबाद छ गगन्या चोर लाई मुख भरी जबाब दिएको मा",
                       "ओली दलाल मुर्दाबाद",
                       "यो गोविन्दे लाई देश निकाला गर्नु पर्छ",
                       "यो मुला गोबिन्द ठिक छैन",
                       "यो पुण्य गौतम जड्या हो जस्तो कस कस लाई लाग्छ ।",
                      "तपाईं कुवा मा दुबेर मरे हुन्छ ।",
                      "अनुहार हेर्दा ठमेल को भालू हो ।"]

results = text_classifier(inference_sentences)


In [ ]:
label_map = {
    0: 'GENERAL POSITIVE',
    1: 'GENERAL NEGATIVE',
    2: 'PROFANITY',
    3: 'VIOLENCE'
}

prediction_results = []
for sent, result in zip(inference_sentences, results):
    pred = result['label'].split('_')[1]
    prediction_results.append([sent, pred, label_map[int(pred)]])

print(tabulate(prediction_results, headers=['Sentences', 'Labels', 'Remarks'], tablefmt='orgtbl'))


## Conclusion

### Sakonii/distilbert-base-nepali


                  precision    recall  f1-score   support

               0       0.75      0.62      0.68        39
               1       0.53      0.62      0.57        61
               2       0.68      0.61      0.65        85
               3       0.56      0.61      0.58        54

        accuracy                           0.62       239
    
       macro avg       0.63      0.62      0.62       239
    weighted avg       0.63      0.62      0.62       239



### xlm-roberta-large

                    precision    recall  f1-score   support

               0       0.00      0.00      0.00        39
               1       0.00      0.00      0.00        61
               2       0.36      1.00      0.52        85
               3       0.00      0.00      0.00        54

        accuracy                           0.36       239
       macro avg       0.09      0.25      0.13       239
    weighted avg       0.13      0.36      0.19       239

### Rajan/NepaliBERT

                  precision    recall  f1-score   support

               0       0.70      0.67      0.68        39
               1       0.48      0.64      0.55        61
               2       0.74      0.59      0.65        85
               3       0.64      0.63      0.64        54

        accuracy                           0.62       239
       macro avg       0.64      0.63      0.63       239
    weighted avg       0.64      0.62      0.63       239